In [1]:
# install ibm watson machine-learning
!pip install -U ibm-watson-machine-learning

In [2]:


import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import os, types
from botocore.client import Config
import ibm_boto3
from ibm_watson_machine_learning import APIClient
import ast
import json

In [3]:
# import data from IBM Cloud Object Storage

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='MX8QVCsnOOw1lOmlWAD2d62le2j7UIRi6V6x0rHNbFPV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = 'datascienceandmlops-donotdelete-pr-lveujb7ul8husb'
object_key = 'empty.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()



,Id,Text,Tags
0,2,aspnet site maps anyone got experience creatin...,"['sql', 'asp.net']"
1,4,adding scripting functionality net application...,"['c#', '.net']"
2,5,use nested classes case working collection cla...,['c++']
3,6,homegrown consumption web services writing web...,['.net']
4,8,automatically update version number would like...,['c#']


In [4]:
# preprocessing

df["Tags"] = df["Tags"].apply(lambda x: ast.literal_eval(x))
y = df["Tags"]

X = df["Text"]

In [8]:
# models

mlb = MultiLabelBinarizer()
multilabel_y = mlb.fit_transform(y)
svc = LinearSVC()
clf = OneVsRestClassifier(svc)

In [9]:
# create pipeline

pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))), 
                    ('tfidf', TfidfTransformer()), 
                    ('clf', OneVsRestClassifier(clf))])


In [10]:
# Use multilabel classes to fit the pipeline.
pipeline.fit(X, multilabel_y)


Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 3))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC())))])

In [11]:
predictions = pipeline.predict(['What is c, python and sql?'])


In [12]:
classes = mlb.inverse_transform(predictions)
print(classes)

[('python', 'sql')]


In [22]:
type(X)

pandas.core.series.Series

In [13]:
wml_credentials = {
    "apikey": 'LhO00vhPZLW0dQ9l96AlTfrQQShRahUsltqnUWodwwuI',
    "url": 'https://eu-gb.ml.cloud.ibm.com',}



In [14]:
client = APIClient(wml_credentials)

In [15]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------------  ------------------------
ID                                    NAME                   CREATED
c5601b64-8b4e-470a-b022-f869d5770ee5  Deploy_stack-overflow  2023-08-27T12:07:41.782Z
------------------------------------  ---------------------  ------------------------


,ID,NAME,CREATED
0,c5601b64-8b4e-470a-b022-f869d5770ee5,Deploy_stack-overflow,2023-08-27T12:07:41.782Z


In [16]:
space_id = 'c5601b64-8b4e-470a-b022-f869d5770ee5'

In [17]:
client.set.default_space(space_id)

'SUCCESS'

In [18]:
MODEL_NAME = 'LinearSVC'
DEPLOYMNET_NAME = 'LinearSVC_Deploy'
BEST_MODEL = clf

In [19]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.2-py3.10")

In [ ]:
metadata = {client.repository.ModelMetaNames.NAME: 'LinearSVC',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid}

published_model = client.repository.store_model(
    model=pipeline,
    meta_props=metadata,
    training_data=X,
    training_target=multilabel_y)

In [ ]:

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:

models_details = client.repository.list_models()


In [ ]:
metadata = {client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit model",
            client.deployments.ConfigurationMetaNames.ONLINE: {}}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)